In [ ]:
# import an image
# crop & CS reconstruction in this script

In [1]:
import hyperspy.api as hys
import numpy as np
import matplotlib.pyplot as plt
import tkinter.filedialog as tkf
from sklearn.decomposition import PCA
import scipy.optimize as spopt
import scipy.fftpack as spfft
from sklearn.linear_model import Lasso
import glob

In [ ]:
whole_img_adr = tkf.askopenfilename()
print(whole_img_adr)

In [ ]:
X_dm = hys.load(whole_img_adr)
X = X_dm.data
print(X.shape)

In [3]:
crop_size = 200

In [ ]:
def pca(data_set, eigenvector=False, projection=None, inverse=False, n_comp=100, solver="auto", whitening=False):
    print("Doing PCA...")

    pca = PCA(n_components=n_comp, svd_solver=solver, whiten=whitening)
    pca.fit(data_set)
    print(pca.explained_variance_ratio_)
    print(np.sum(pca.explained_variance_ratio_))

    fig1, ax1 = plt.subplots(figsize=(5,3))
    ax1.plot(np.cumsum(pca.explained_variance_ratio_), c="darkslategrey")
    ax1.grid()
    ax1.set_xlabel('number of components')
    ax1.set_ylabel('cumulative explained variance')
    #print len(pca.components_)
    #plt.show()
    #plt.close(fig1)

    reduced_projection = pca.transform(data_set)
    restore = pca.inverse_transform(reduced_projection)
        
    if eigenvector:
        fig2, ax2 = plt.subplots(n_comp, 1, figsize=(10, n_comp))
        temp_flat = ax2.flat
        for i, ax in enumerate(temp_flat[:n_comp]):
            ax.plot(pca.components_[i], c="darkslategrey")
            ax.grid()
        #plt.show()
        #plt.close(fig2)

    if projection:
        fig3= plt.figure(figsize=(10,10))
        ax3 = fig3.add_subplot(221)
        ax3.scatter(reduced_projection[:, 0], reduced_projection[:, 1], s=1.5, c="darkslategrey")
        ax3.grid()
        ax3.set_xlabel("component 1")
        ax3.set_ylabel("component 2")

        ax4 = fig3.add_subplot(222)
        ax4.scatter(reduced_projection[:, 1], reduced_projection[:, 2], s=1.5, c="darkslategrey")
        ax4.grid()
        ax4.set_xlabel("component 2")
        ax4.set_ylabel("component 3")

        ax5 = fig3.add_subplot(223)
        ax5.scatter(reduced_projection[:, 0], reduced_projection[:, 2], s=1.5, c="darkslategrey")
        ax5.grid()
        ax5.set_xlabel("component 1")
        ax5.set_ylabel("component 3")
        
        if n_comp > 2:
            ax6 = fig3.add_subplot(224, projection="3d")
            ax6.scatter(reduced_projection[:, 0], reduced_projection[:, 1],reduced_projection[:, 2], zdir="z", s=1.5, c="darkslategrey")
            ax6.grid()
            ax6.set_xlabel("component 1")
            ax6.set_ylabel("component 2")
            ax6.set_zlabel("component 3")
        
        #plt.show()
        #plt.close(fig3)

    if inverse:
        n2 = 10
        fig4, ax7 = plt.subplots(n2, 2, figsize=(10,20))
        for i in range(n2):
            ax7[i, 0].plot(data_set[i], c="blue")
            ax7[i, 0].grid()
            ax7[i, 1].plot(restore[i], c="crimson")
            ax7[i, 1].grid()
        #fig4.tight_layout()
        #plt.show()
        #plt.close(fig4)
        
    return pca.components_, pca.mean_, reduced_projection, restore

In [ ]:
comp_vectors_pca, mean_pca, projections_pca, restoration_pca = pca(X, n_comp=10, 
                                                                   eigenvector=True, 
                                                                   projection=False, 
                                                                   inverse=False,
                                                                  whitening=False)
plt.show()

In [ ]:
# PCA : reconstruction

num_comp = 10
comp_array = np.arange(num_comp) # or you can create manually

reconstructed_pca = np.dot(projections_pca[:, comp_array], comp_vectors_pca[comp_array]) + mean_pca
        
print(np.max(X))
print(np.max(reconstructed_pca))
fig, ax = plt.subplots(1, 2, figsize=(20, 20))
ax[0].imshow(X[:100, :100], cmap="afmhot")
ax[0].axis("off")
ax[1].imshow(reconstructed_pca[:100, :100], cmap="afmhot")
ax[1].axis("off")
plt.show()

In [ ]:
threshold = 0.60
normalized = reconstructed_pca / np.max(reconstructed_pca)
normalized[np.where(normalized < threshold)] = 0
normalized[np.where(normalized != 0)] = 1

In [ ]:
print(np.sum(normalized)/normalized.size)

fig, ax = plt.subplots(1, 2, figsize=(20, 20))
ax[0].imshow(reconstructed_pca[:100, :100], cmap="afmhot")
ax[0].axis("off")
ax[1].imshow(normalized[:100, :100], cmap="afmhot")
ax[1].axis("off")
plt.show()

In [ ]:
phi = normalized.copy()

In [4]:
def dct2d(x):
    return spfft.dct(spfft.dct(x.T, norm='ortho', axis=0).T, norm='ortho', axis=0)

def idct2(x):
    return spfft.idct(spfft.idct(x.T, norm='ortho', axis=0).T, norm='ortho', axis=0)

In [ ]:
dctrow = spfft.idct(np.identity(crop_size), norm='ortho', axis=0)
dctcol = spfft.idct(np.identity(crop_size), norm='ortho', axis=0)
    
A = np.kron(dctrow, dctcol)
print(A.shape)
del dctrow
del dctcol
    
lasso = Lasso(alpha=0.00001, max_iter=2000, tol=0.00001, random_state=56)

sliced = whole_img[0:crop_size, 0:crop_size]
phi_prime = phi[0:crop_size, 0:crop_size]
ri = np.where(phi_prime==1.0)[0]
A_prime = A[ri, :]

In [ ]:
b = sliced.T.flat[ri]
b = np.expand_dims(b, axis=1)
lasso.fit(A_prime, b)
Xa = idct2(np.array(lasso.coef_).reshape(sy, sx).T)
fig, ax = plt.subplots(1, 2, figsize=(20, 20))
ax[0].imshow(sliced, cmap="afmhot")
ax[0].axis("off")
ax[1].imshow(Xa, cmap="afmhot")
ax[1].axis("off")
plt.show()